In [53]:
import pandas as pd
import numpy as np
import psycopg2
import os
from dotenv import load_dotenv
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
load_dotenv()

True

In [54]:
# Equation for mean average percentage error for test the model performance
def mape(y_true, y_pred):
    ape = np.abs((y_true - y_pred) / y_true)
    return np.mean(ape)

def wmape(y_true, y_pred):
    return np.sum(np.abs((y_true - y_pred))) / np.sum(np.abs(y_true))

In [55]:
# Database connection and query
host = os.getenv("HOST")
dbname = os.getenv("DATABASE_NAME")
user = os.getenv("USERNAME")
password = os.getenv("DATABASE_PASSWORD")

conn_string = "host={0} user={1} dbname={2} password={3}".format(host, user, dbname, password)
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

df_productsales = pd.read_sql("SELECT * FROM core_productsale;", conn)
df_products = pd.read_sql("SELECT * FROM core_product;", conn)
df_sales = pd.read_sql("SELECT * FROM core_sale;", conn)
df_products

/home/francisco/workspace/lambda-system/machine-learning-dicascada/venv/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/francisco/workspace/lambda-system/machine-learning-dicascada/venv/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/francisco/workspace/lambda-system/machine-learning-dicascada/venv/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn

,id,name,description,presentation,cost,price_1,price_2,price_3,brand,category_id,image,cost_currency,price_1_currency,price_2_currency,price_3_currency,code
0,1,REQUESON DURO CON SAL (E),Nan,KG,1.10,1.42,1.30,1.38,Yaracal,23,https://i.ibb.co/SrMrfyV/pngwing-com.png,USD,USD,USD,USD,000005
1,2,SUERO DE LECHE MIS QUESOS 700 ML,Nan,UND,1.07,1.47,1.25,1.33,Mis quesos,8,https://i.ibb.co/SrMrfyV/pngwing-com.png,USD,USD,USD,USD,00001
2,3,COMPRA MATERIALES Y EQUIPOS DE OFICINA,Materiales y equipos de oficina,NAN,0.00,0.00,0.00,0.00,Nan,1056,https://i.ibb.co/SrMrfyV/pngwing-com.png,USD,USD,USD,USD,00003
3,4,QUESO PASTEURIZADO PURISIMA EL TUNAL (E),Queso blaco blando extragraso,KG,4.95,6.04,5.69,5.89,El tunal,23,https://i.ibb.co/SrMrfyV/pngwing-com.png,USD,USD,USD,USD,00004
4,5,CREMOSITO CHEDDAR TINA 220 GRS DERILAC,Cremosito cheddar 220 grs,UND,1.50,1.90,1.74,1.84,Derilac,8,https://i.ibb.co/SrMrfyV/pngwing-com.png,USD,USD,USD,USD,00005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,485,YOGURT GRIEGO 16 OZ NATUGURT,Nan,UND,3.00,3.90,3.80,3.90,Natugurt,1063,https://i.ibb.co/SrMrfyV/pngwing-com.png,USD,USD,USD,USD,YO022
485,486,YOGURT NATURAL 16 OZ NATUGURT,Nan,UND,2.00,2.80,2.70,2.80,Natugurt,1063,https://i.ibb.co/SrMrfyV/pngwing-com.png,USD,USD,USD,USD,YO023
486,487,YOGURT DE COCO NATUGURT,Nan,UND,3.00,3.90,3.80,3.90,Natugurt,1063,https://i.ibb.co/SrMrfyV/pngwing-com.png,USD,USD,USD,USD,YO024
487,488,QUESO DE YOGURT 16 OZ NATUGURT,Nan,UND,3.00,4.00,3.80,3.90,Natugurt,1063,https://i.ibb.co/SrMrfyV/pngwing-com.png,USD,USD,USD,USD,YO025


In [56]:
df_productsales

,id,quantity,income,product_id,sale_id,income_currency
0,16921,4.10,14.35,17,00034279,USD
1,16953,4.00,14.00,17,00034290,USD
2,16930,8.10,43.52,22,00034282,USD
3,17005,6.80,21.87,23,00034311,USD
4,16906,24.00,46.72,33,00034269,USD
...,...,...,...,...,...,...
71828,18772,25.30,0.00,489,00034825,USD
71829,19432,4.30,0.00,489,00035023,USD
71830,19630,8.70,0.00,489,00035079,USD
71831,59878,4.40,0.00,489,00001021,USD


In [57]:
df_merged = pd.merge(df_productsales, df_products, left_on="product_id", right_on="id", suffixes=("_productsale", "_product"))
df_merged.drop(columns=["brand", "image", "cost_currency", "price_1_currency", "price_2_currency", "price_3_currency", "code", "price_1", "price_2", "price_3", "description", "presentation", "income_currency"], inplace=True)
df_merged.set_index("id_product", inplace=True)
df_merged

,id_productsale,quantity,income,product_id,sale_id,name,cost,category_id
id_product,,,,,,,,
17,16921,4.1,14.35,17,00034279,QUESO TELITA CACHAPERO (E),2.70,23
17,16953,4.0,14.00,17,00034290,QUESO TELITA CACHAPERO (E),2.70,23
17,17217,3.6,12.60,17,00034379,QUESO TELITA CACHAPERO (E),2.70,23
17,17330,3.4,11.90,17,00034415,QUESO TELITA CACHAPERO (E),2.70,23
17,17334,3.4,11.90,17,00034418,QUESO TELITA CACHAPERO (E),2.70,23
...,...,...,...,...,...,...,...,...
6,59532,6.0,16.50,6,004428,ACEITUNAS RE/PIM ENV. 450GR ESCURRIDO,2.75,54
6,81979,11.0,30.25,6,NE010339,ACEITUNAS RE/PIM ENV. 450GR ESCURRIDO,2.75,54
6,82805,6.0,16.50,6,NE010715,ACEITUNAS RE/PIM ENV. 450GR ESCURRIDO,2.75,54


In [76]:
new_index = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
df_merged_sales_with_products = pd.merge(df_merged, df_sales, left_on="sale_id", right_on="id", suffixes=("_productsale", "_sale"))
df_merged_sales_with_products["year"] = df_merged_sales_with_products.apply(lambda row: row["date"].year, axis=1)
df_merged_sales_with_products["month"] = df_merged_sales_with_products.apply(lambda row: row["date"].month, axis=1)
df_merged_sales_with_products.drop(columns=["id", "description", "income_currency", "status", "date" ], inplace=True)
df_merged_sales_with_products.set_index(['month', 'product_id'])
df_merged_sales_with_products.reindex(new_index, fill_value=0)
df_merged_sales_with_products.reset_index(inplace=True)
df_merged_sales_with_products[df_merged_sales_with_products["product_id"] == 1]

,index,id_productsale,quantity,income_productsale,product_id,sale_id,name,cost,category_id,income_sale,client_id,salesman_id,year,month
676,676,59792,16.40,41.05,9,00001005,QUESO BLANCO DE CABRA (E),2.3,23,205.89,1022,1,2018,12
682,682,60050,9.60,24.03,9,00001142,QUESO BLANCO DE CABRA (E),2.3,23,348.46,1022,1,2019,5
691,691,60059,9.60,24.03,9,00001143,QUESO BLANCO DE CABRA (E),2.3,23,356.46,1022,1,2019,5
718,718,60620,8.84,22.13,9,00001504,QUESO BLANCO DE CABRA (E),2.3,23,362.28,1022,1,2019,10
817,817,56367,0.45,1.13,9,00003225,QUESO BLANCO DE CABRA (E),2.3,23,634.80,1022,1,2020,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67690,67690,18914,7.50,18.78,9,00034872,QUESO BLANCO DE CABRA (E),2.3,23,18.78,138,2,2019,4
67691,67691,19509,2.30,5.76,9,00035043,QUESO BLANCO DE CABRA (E),2.3,23,146.59,610,2,2019,5
67697,67697,20092,6.00,15.02,9,00035229,QUESO BLANCO DE CABRA (E),2.3,23,15.02,247,4,2019,6
67698,67698,26177,45.67,114.33,9,00037015,QUESO BLANCO DE CABRA (E),2.3,23,114.33,752,7,2020,1


In [59]:
# grouping products per month for return stadistics about this.
# This data frame will be used to plot the graphs for see the historic of sales per product month in every year
# This data frame can be use for trimester too

df_groupby_products = df_merged_sales_with_products.groupby(["year", "month", "product_id"])
sales_data = {
    'income': df_groupby_products['income_productsale'].sum(),
    'count': df_groupby_products['quantity'].sum(),
    'name': df_groupby_products['name'].first(),
}

df_sales_per_months = pd.DataFrame(sales_data)
df_sales_per_months

income   count  \
year month product_id                    
2018 11    9             84.36   33.70   
           12           114.45   31.50   
           13           660.02  108.20   
           17            54.95   15.70   
           22           296.61   55.20   
...                        ...     ...   
2022 2     438           77.99   12.16   
           439         1091.26  213.00   
           440         2218.03  741.00   
           443          207.95   56.00   
           445         2405.48  847.00   

                                                           name  
year month product_id                                            
2018 11    9                          QUESO BLANCO DE CABRA (E)  
           12                           QUESO BLANCO DE RES (E)  
           13           QUESO BLANCO PASTEURIZADO MIS QUESOS(E)  
           17                        QUESO TELITA CACHAPERO (E)  
           22                      QUESO BLANCO TIPO TELITA (E)  
...                                                         ...  
2022 2     438               SALCHICHA VIENA GRANEL X KG ALIMEX  
           439          SALCHICHA VIENA 800 GRS X UNIDAD ALIMEX  
           440           SALCHICHA VIENA 450GRS X UNIDAD ALIMEX  
           443                   SALCHICHA WIENNER 450GR ALIMEX  
           445         SALCHICHA DE POLLO 450GR ALIMEX EL TUNAL  

[4444 rows x 3 columns]

In [74]:
# Copy df for ML model. The other DF is for chart.
#Then, it just set split point, reset index and
# split the dataframe in two parts, one for train and one for test

df_machine_learning = df_sales_per_months.copy()
df_machine_learning.reset_index(inplace=True)
split_point = 2021
df_machine_learning.drop(df_machine_learning[df_machine_learning['name'] == "NONE"].index, inplace=True)

data_train = df_machine_learning[df_machine_learning['year'] < split_point].copy()
data_valid = df_machine_learning[df_machine_learning['year'] >= split_point].copy()
data_train

data_valid[data_valid['product_id'] == 9]

,year,month,product_id,income,count,name
2994,2021,6,9,63.79,25.48,QUESO BLANCO DE CABRA (E)
3113,2021,7,9,66.57,26.59,QUESO BLANCO DE CABRA (E)


In [61]:
# Add the columns which gonna be predicted.
# It just set the products sales next month using 
# the income of the next products sales in the DF

data_train["sales_next_month"] = data_train.groupby("product_id")["income"].shift(-1)
data_train["sales_next_month_count"] = data_train.groupby("product_id")["count"].shift(-1)
data_valid["sales_next_month"] = data_valid.groupby("product_id")["income"].shift(-1)
data_valid["sales_next_month_count"] = data_valid.groupby("product_id")["count"].shift(-1)

In [62]:
# Drop NaN values for train, otherwise, the code dont work
# Add lag sales next day in both DF train and valid. It is use
# when some info doesnt exist or there has missing values

data_train.dropna(inplace=True)
data_train["lag_sales_next_month"] = data_train.groupby("product_id")["income"].shift(1)
data_train["lag_sales_next_month_count"] = data_train.groupby("product_id")["count"].shift(1)
data_valid["lag_sales_next_month"] = data_valid.groupby("product_id")["income"].shift(1)
data_valid["lag_sales_next_month_count"] = data_valid.groupby("product_id")["count"].shift(1)

In [63]:
# Set diff between sales and sales next day

data_valid["diff_sales_next_month"] = data_valid.groupby("product_id")["income"].diff(1)
data_valid["diff_sales_next_month_count"] = data_valid.groupby("product_id")["count"].diff(1)
data_train["diff_sales_next_month"] = data_train.groupby("product_id")["income"].diff(1)
data_train["diff_sales_next_month_count"] = data_train.groupby("product_id")["count"].diff(1)
data_train

,year,month,product_id,income,count,name,sales_next_month,sales_next_month_count,lag_sales_next_month,lag_sales_next_month_count,diff_sales_next_month,diff_sales_next_month_count
0,2018,11,9,84.36,33.70,QUESO BLANCO DE CABRA (E),357.72,142.90,NaN,NaN,NaN,NaN
1,2018,11,12,114.45,31.50,QUESO BLANCO DE RES (E),1104.17,303.90,NaN,NaN,NaN,NaN
2,2018,11,13,660.02,108.20,QUESO BLANCO PASTEURIZADO MIS QUESOS(E),861.93,141.30,NaN,NaN,NaN,NaN
3,2018,11,17,54.95,15.70,QUESO TELITA CACHAPERO (E),23.80,6.80,NaN,NaN,NaN,NaN
4,2018,11,22,296.61,55.20,QUESO BLANCO TIPO TELITA (E),441.15,82.10,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2232,2020,11,449,107.06,74.00,SUERO DE LECHE YARACAL BOLSA 1KG,4.34,3.00,21.71,15.00,85.35,59.00
2233,2020,11,452,1903.79,252.60,MORTADELA TAPARA 1KG ALIMEX (E),808.92,107.33,691.10,91.70,1212.69,160.90
2234,2020,11,456,453.07,76.79,JAMÏ¿½N COCIDO PIERNA VIGOR,69.50,11.78,104.90,17.78,348.17,59.01
2235,2020,11,458,310.29,66.68,ESPALDA COCIDA VIGOR,23.96,5.15,NaN,NaN,NaN,NaN


In [64]:
# Baseline for indicators for precission of the model

data_train_filtered = data_train.copy()
data_train_filtered.reset_index(inplace=True)
data_train_filtered.sort_values(inplace=True, by=["product_id", "year", "month"])


#y_pred = data_train_filtered["income"]
#y_true = data_train_filtered["sales_next_month"]

y_pred = data_train["income"]
y_true = data_train["sales_next_month"]


print(mape(y_true, y_pred))
wmape(y_true, y_pred)



3.172066005858639


0.5638986031245905

In [65]:
# Set features for ML model, which columns will be use for predict
# Imputer is for transform some data
# Xtr and Ytr are the X and Y values for train the model
# Then the model is just a RandomForestRegressor and with fit it get trained

features = ["income", "count"]
imputer = SimpleImputer()
# Xtr_per_month = imputer.fit_transform(data_train_filtered[features])
Xtr_per_month = imputer.fit_transform(data_train[features])
ytr_per_month = data_train['sales_next_month']


model = RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=6)
model.fit(Xtr_per_month, ytr_per_month)

RandomForestRegressor(n_jobs=6, random_state=0)

In [66]:
# Predicting sales for everyh day with the model
# This model just predict the income of the next month, not how many sales

# The final data frame is just for seing true values and predicted values

data_valid_filtered = data_valid.copy()
data_valid_filtered.reset_index(inplace=True)

#Xval_per_month = imputer.fit_transform(data_valid_filtered[features])
#yval_per_month = data_valid_filtered['sales_next_month']

Xval_per_month = imputer.fit_transform(data_valid[features])
yval_per_month = data_valid['sales_next_month']


prediction_per_month = model.predict(Xval_per_month)

df_forest_per_month = pd.DataFrame({"product_id": data_valid["product_id"],"product": data_valid["name"],"value": data_valid["income"],"year": data_valid["year"], "month": data_valid["month"] ,"predicted": prediction_per_month, "true": yval_per_month, "diff": data_valid["diff_sales_next_month"]})
df_forest_per_month.sort_values(inplace=True, by=["product_id"])
df_forest_per_month

,product_id,product,value,year,month,predicted,true,diff
3111,1,REQUESON DURO CON SAL (E),39.23,2021,7,50.865500,NaN,NaN
2646,4,QUESO PASTEURIZADO PURISIMA EL TUNAL (E),179.89,2021,3,341.505700,78.70,NaN
2767,4,QUESO PASTEURIZADO PURISIMA EL TUNAL (E),78.70,2021,4,96.845700,192.65,-101.19
3986,4,QUESO PASTEURIZADO PURISIMA EL TUNAL (E),192.65,2021,12,332.366000,NaN,113.95
3286,5,CREMOSITO CHEDDAR TINA 220 GRS DERILAC,215.53,2021,8,403.343400,NaN,188.13
...,...,...,...,...,...,...,...,...
3281,480,ADEREZO DE CELERY NATUGURT,6.50,2021,7,99.469307,1.30,NaN
3445,480,ADEREZO DE CELERY NATUGURT,1.30,2021,8,21.902500,NaN,-5.20
3282,481,ADEREZO DE CILANTRO NATUGURT,6.50,2021,7,99.469307,NaN,NaN
3283,482,ADEREZO DE TOCINETA NATUGURT,6.50,2021,7,99.469307,NaN,NaN


In [67]:
# see the percentage for the error of the model for sales income

print(mape(df_forest_per_month[df_forest_per_month["product_id"] == 20]["true"], df_forest_per_month[df_forest_per_month["product_id"] == 20]["predicted"]))
print(wmape(df_forest_per_month[df_forest_per_month["product_id"] == 20]["true"], df_forest_per_month[df_forest_per_month["product_id"] == 20]["predicted"]))

0.8765376869065673
0.6575469915755171


In [68]:
# Baseline for indicators for precission of the model
# This baseline is for how many sales will be

y_pred_count = data_train_filtered["sales_next_month_count"]
y_true_count = data_train_filtered['count']

print(mape(y_true, y_pred))
wmape(y_true, y_pred)

3.172066005858639


0.5638986031245905

In [69]:
# Set features for ML model, which columns will be use for predict
# Imputer is for transform some data
# Xtr and Ytr are the X and Y values for train the model
# Then the model is just a RandomForestRegressor and with fit it get trained
# Again, same shit, but for sales count

features_count = ["income", "count"]
imputer_count = SimpleImputer()
Xtr_per_month_count = imputer_count.fit_transform(data_train[features_count])
ytr_per_month_count = data_train['sales_next_month_count']


model_count = RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=6)
model_count.fit(Xtr_per_month_count, ytr_per_month_count)

RandomForestRegressor(n_jobs=6, random_state=0)

In [70]:
# Predicting sales for everyh day with the model
# This model just predict the income of the next month, not how many sales

# The final data frame is just for seing true values and predicted values


Xval_per_month_count = imputer.fit_transform(data_valid[features])
yval_per_month_count = data_valid['sales_next_month_count']

prediction_per_month_count = model_count.predict(Xval_per_month_count)

df_forest_per_month_count = pd.DataFrame({"value": data_valid['count'],"product_id": data_valid["product_id"] ,"product": data_valid["name"],"year": data_valid["year"], "month": data_valid["month"] ,"predicted": prediction_per_month_count, "true": yval_per_month_count, "diff_count": data_valid["diff_sales_next_month_count"]})
df_forest_per_month_count
df_forest_per_month_count[df_forest_per_month["product_id"] == 20]

/tmp/ipykernel_4031/2995192586.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_forest_per_month_count[df_forest_per_month["product_id"] == 20]


,value,product_id,product,year,month,predicted,true,diff_count
2651,1610.0,20,SALCHICHA VIENA 225GR ALIMEX,2021,3,1041.7700,338.0,NaN
2772,338.0,20,SALCHICHA VIENA 225GR ALIMEX,2021,4,190.4093,262.0,-1272.0
2886,262.0,20,SALCHICHA VIENA 225GR ALIMEX,2021,5,189.3043,432.0,-76.0
3000,432.0,20,SALCHICHA VIENA 225GR ALIMEX,2021,6,601.1415,566.0,170.0
3119,566.0,20,SALCHICHA VIENA 225GR ALIMEX,2021,7,541.4905,438.0,134.0
3291,438.0,20,SALCHICHA VIENA 225GR ALIMEX,2021,8,742.3199,527.0,-128.0
3450,527.0,20,SALCHICHA VIENA 225GR ALIMEX,2021,9,429.3462,2048.0,89.0
3623,2048.0,20,SALCHICHA VIENA 225GR ALIMEX,2021,10,1933.1000,1652.0,1521.0
3801,1652.0,20,SALCHICHA VIENA 225GR ALIMEX,2021,11,3492.4500,1097.0,-396.0
3992,1097.0,20,SALCHICHA VIENA 225GR ALIMEX,2021,12,1389.8989,1153.0,-555.0


In [71]:
print(mape(df_forest_per_month_count[df_forest_per_month["product_id"] == 20]["true"], df_forest_per_month[df_forest_per_month_count["product_id"] == 20]["predicted"]))
print(wmape(df_forest_per_month_count[df_forest_per_month["product_id"] == 20]["true"], df_forest_per_month[df_forest_per_month_count["product_id"] == 20]["predicted"]))

1.978586429885536
1.476963706896551


/tmp/ipykernel_4031/1951812217.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(mape(df_forest_per_month_count[df_forest_per_month["product_id"] == 20]["true"], df_forest_per_month[df_forest_per_month_count["product_id"] == 20]["predicted"]))
/tmp/ipykernel_4031/1951812217.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(wmape(df_forest_per_month_count[df_forest_per_month["product_id"] == 20]["true"], df_forest_per_month[df_forest_per_month_count["product_id"] == 20]["predicted"]))
